In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
import tensorflow as tf
# import gym
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv


In [6]:
# Descargar los precios
data = yf.download('AAPL', start='2011-10-21', end='2023-05-24', interval="1d")

prices = data['Adj Close'].values
prices2 = prices[54:]
# Imprimir data
#print(data.head())

[*********************100%%**********************]  1 of 1 completed


In [ ]:

# Función para calcular el RSI
def calculate_rsi(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n + 1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = [np.nan] * (n - 1) + [100. - 100. / (1. + rs)]

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # Cambio actual
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        rs = up / down
        rsi.append(100. - 100. / (1. + rs))

    return rsi



# Calcular el RSI
rsi = calculate_rsi(prices)
rsi2 = rsi[40:]

In [ ]:
# Definir el entorno para el agente de RL
class CustomEnv(gym.Env):
    def __init__(self, prices2, rsi2):
        super(CustomEnv, self).__init__()
        self.data = prices2
        self.predictions = rsi2
        self.current_step = 0
        self.done = False
        self.reward_range = (-1, 1)
        self.action_space = gym.spaces.Discrete(3)
        self.precio_anterior = 0
        self.positions = []  # Lista para almacenar las posiciones anteriores
        self.total_reward = 0  # Variable para rastrear la suma total de las recompensas en un episodio
        self.compras_realizadas = []

        # Definir el espacio de observación
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(2, len(self.predictions)))

    def reset(self):
        self.current_step = 0
        self.total_reward = 0
        self.done = False
        self.positions = []  # Reiniciar la lista de posiciones anteriores

        # Devolver la observación inicial directamente
        return self.get_observation()

    def step(self, action):
        # Tomar la acción y actualizar el entorno
        # Calcula la recompensa en función de la acción y el estado actual
        reward = self.calculate_reward(action)

        # Actualizar la suma total de recompensas en el episodio actual
        self.total_reward += reward

        # Actualizar el estado interno y verificar si el episodio ha terminado
        self.current_step += 1
        if self.current_step == len(self.data):
            self.done = True

        # Devolver la observación, la recompensa, si el episodio ha terminado y cualquier información adicional
        return self.get_observation(), reward, self.done, {}




    def calculate_reward(self, action):
        reward = 0  # Valor predeterminado de recompensa

        if self.current_step >= len(self.data):
            self.current_step = len(self.data) - 1

        # Obtener el precio actual, la posición anterior y el valor RSI actual
        current_price = self.data[self.current_step]
        current_rsi = self.predictions[self.current_step]

        # Calcular la ganancia o pérdida según la acción tomada
        #if action == 0:  # Acción: no hacer nada
           # reward = 0
        if action == 0:  # Acción: comprar
            if current_rsi < 40:
                self.positions.append(current_price)
                reward = 0  # Costo de comprar
        elif action == 1:  # Acción: vender
            if len(self.positions) > 0 and current_rsi > 65:
                num_positions = len(self.positions)  # Número de posiciones previas

                # Calcular el valor total de la venta (suma de todos los precios de compra)
                total_sale_value = sum(self.positions)

                if total_sale_value > 0:
                    reward = (current_price * num_positions - total_sale_value) / total_sale_value
                else:
                    reward = 0  # Penalización por pérdida

                # Reiniciar las posiciones después de la venta
                self.positions = []

        return reward

    def get_observation(self):
        if self.current_step >= len(self.data):
            self.current_step = len(self.data) - 1

        current_price = self.data[self.current_step]
        current_prediction = self.predictions[self.current_step]

        # Crear una observación con forma (2, 621) concatenando el precio y la predicción
        observation = np.zeros((2, len(self.predictions)))
        observation[0, self.current_step] = current_price
        observation[1, self.current_step] = current_prediction

        return observation

    def get_total_reward(self):
        return self.total_reward
# Crear el entorno y envolverlo en DummyVecEnv
env = DummyVecEnv([lambda: CustomEnv(prices2, rsi2)])
# Crear y entrenar el agente RL con PPO
model_rl = PPO("MlpPolicy", env, verbose=1)
model_rl.learn(total_timesteps=50000)


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 966  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 449         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.064628355 |
|    clip_fraction        | 0.632       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.07       |
|    explained_variance   | -37.1       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.159      |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.115      |
|    value_loss           | 0.0495      |
-----------------------------------------
-----------------

In [ ]:
def evaluate_agent(agent, eval_episodes=20):
    total_rewards = []
    total_compras = 0
    total_ventas = 0
    total_transacciones_ganadoras = 0  # Variable para contar transacciones con recompensa > 0

    for i in range(eval_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        num_compras = 0
        num_ventas = 0
        num_transacciones_ganadoras = 0  # Contador para el episodio actual

        while not done:
            action, _ = agent.predict(obs)
            obs, reward, done, _ = env.step(action)
            total_reward += reward

            if action == 1:
                num_compras += 1
            elif action == 2:
                num_ventas += 1

            if reward > 0:  # Incrementar el contador si la recompensa es positiva
                num_transacciones_ganadoras += 1

        total_rewards.append(total_reward)
        total_compras += num_compras
        total_ventas += num_ventas
        total_transacciones_ganadoras += num_transacciones_ganadoras

        #print(f"Episode {i+1}: Compras = {num_compras}, Ventas = {num_ventas}, "
         #     f"Reward = {total_reward}, Transacciones Ganadoras = {num_transacciones_ganadoras}")

    avg_reward = np.mean(total_rewards)
    avg_compras = total_compras / eval_episodes
    avg_ventas = total_ventas / eval_episodes
    avg_transacciones_ganadoras = total_transacciones_ganadoras / eval_episodes
    avg_transacciones_ganadoras_per_compra = avg_transacciones_ganadoras / avg_compras

    print(f"Promedio reward en {eval_episodes} episodios: {avg_reward}")
    print(f"Promedio de compras: {avg_compras}")
    print(f"Promedio de ventas: {avg_ventas}")
    #print(f"Promedio de Transacciones Ganadoras: {avg_transacciones_ganadoras}")
    #print(f"Promedio de Transacciones Ganadoras por Compra: {avg_transacciones_ganadoras_per_compra}")

# Evaluar el agente entrenado
evaluate_agent(model_rl)

Promedio reward en 20 episodios: 2.7476978302001953
Promedio de compras: 979.9
Promedio de ventas: 899.55


In [ ]:
def evaluate_agent(agent, eval_episodes=10):
    total_rewards = []
    total_compras = 0
    total_ventas = 0
    total_num_operaciones_ganadoras = 0
    total_num_operaciones_perdedoras = 0
    for i in range(eval_episodes):  # Agregar la variable 'i' en el bucle
        obs = env.reset()
        done = False
        total_reward = 0
        num_compras = 0
        num_ventas = 0
        num_operaciones_ganadoras = 0
        num_operaciones_perdedoras= 0

        while not done:
            action, _ = agent.predict(obs)
            obs, reward, done, _ = env.step(action)
            total_reward += reward

            if action == 1:
                num_compras += 1
            elif action == 2:
                num_ventas += 1

            if reward > 0:
                num_operaciones_ganadoras += 1

            if reward < 0:
                num_operaciones_perdedoras += 1





        total_rewards.append(total_reward)
        total_compras += num_compras
        total_ventas += num_ventas
        total_num_operaciones_ganadoras += num_operaciones_ganadoras
        total_num_operaciones_perdedoras += num_operaciones_perdedoras

        print(f"Episode {i+1}: Compras = {num_compras}, Ventas = {num_ventas}, Reward = {total_reward},Número de operaciones ganadoras = {num_operaciones_ganadoras}, Número de operaciones perdedoras = {num_operaciones_perdedoras}")

    avg_reward = np.mean(total_rewards)
    avg_compras = total_compras / eval_episodes
    avg_ventas = total_ventas / eval_episodes
    avg_num_operaciones_ganadoras = total_num_operaciones_ganadoras / eval_episodes
    avg_num_operaciones_perdedoras = total_num_operaciones_perdedoras / eval_episodes
    print(f"Promedio reward  {eval_episodes} episodes: {avg_reward}")
    print(f"Promedio de compras: {avg_compras}")
    print(f"Promedio de ventas: {avg_ventas}")
    print(f"Promedio Numero acciones ganadoras: {avg_num_operaciones_ganadoras}")
    print(f"Promedio Numero acciones perdedoras: {avg_num_operaciones_perdedoras}")

# Evaluar el agente entrenado
evaluate_agent(model_rl)

Episode 1: Compras = 108, Ventas = 103, Reward = [0.14611207],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 2: Compras = 109, Ventas = 111, Reward = [0.15833612],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 3: Compras = 117, Ventas = 109, Reward = [0.17216119],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 4: Compras = 110, Ventas = 106, Reward = [0.20293912],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 5: Compras = 125, Ventas = 94, Reward = [0.20413086],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 6: Compras = 109, Ventas = 118, Reward = [0.19923548],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 7: Compras = 114, Ventas = 108, Reward = [0.13213387],Número de operaciones ganadoras = 2, Número de operaciones perdedoras = 0
Episode 8: Compras = 117, Ventas = 105, Reward = 

In [ ]:
# Evaluar el agente entrenado
def evaluate_agent(agent, eval_episodes=10):
    total_rewards = []
    for _ in range(eval_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = agent.predict(obs)
            obs, reward, done, _ = env.step(action)
            total_reward += reward
        total_rewards.append(total_reward)
    avg_reward = np.mean(total_rewards)
    print(f"Average reward over {eval_episodes} episodes: {avg_reward}")

# Evaluar el agente entrenado
evaluate_agent(model_rl)

Average reward over 10 episodes: 0.18665124475955963
